# Функции: избранные темы

## Вступление: контекстные менеджеры

В Python по умолчанию задана некоторая точность вычисления чисел с плавающей точкой, которая, что логично, может отличаться от требуемой для решения конкретной задачи точности. Для пример, будет рассматривать точность вычислений в библиотеке **decimal**.

Предположим, нас устраивает точность вычислений по умолчанию в этой библиотеке, но для некоторых расчетов мы бы хотели использовать повышенную или пониженную точность и только для них. Как нам быть? Тут-то нам на помощь и придут контекстные менеджеры.

В этом примере мы поговорим о контекстных менеджерах и напишем свой менеджер для установки точности вычислений, которые будет работать примерно так:

```python
with set_precision(5):
    ...
```

In [3]:
from typing import Generator
from decimal import getcontext
from contextlib import contextmanager

@contextmanager
def set_precision(precision: int) -> Generator[None, None, None]:
    precision = int(precision)

    if precision < 1:
        ValueError('precision should be positive integer')

    # сохранение контекста
    precision_old = getcontext().prec
    getcontext().prec = precision

    # возвращение управления
    try:
        yield

    # восстановление контекста
    finally:
        getcontext().prec = precision_old
        print('restore precision')

In [12]:
with set_precision(54):
    print(3.14/2.72)

1.1544117647058822


## Разминка: геометрическая прогрессия

Напишите бесконечный генератор геометрической прогрессии. В качестве параметров генератор должен принимать:  
- первый член прогрессии
- шаг прогрессии

In [15]:
from typing import Generator


def generate_geometry_progression(
    start: float = 1,
    step: float = 1
) -> Generator[float, None, None]:
    current_member, step = float(start), float(step)

    while True:
        yield current_member 
        current_member *= step

In [16]:
generator = generate_geometry_progression(step = 0.5)
for i in range(5):
    print(next(generator))

1.0
0.5
0.25
0.125
0.0625


## Задание 1: float_range

Необходимо создать аналог range(), который генерирует арифметическую прогрессию. Ниже приведены примеры использования:

*Пример 1*:

```python
for i in float_range(stop=5):
    print(i)
```

*Вывод*:
```console
0
1
2
3
4
```
___
*Пример 2*:

```python
for i in float_range(stop=1, step=0.5):
    print(i)
```

*Вывод*:
```console
0
0.5
```
___
*Пример 3*:

```python
for i in float_range(start=1, stop=-1, step=-0.5):
    print(i)
```

*Вывод*:
```console
1
0.5
0
-0.5
```

In [12]:
from typing import Generator


def float_range(
    start = 0,
    stop = 1,
    step = 1
) -> Generator[float, None, None]:

    current_number, stop, step = float(start), float(stop), float(step)

    if ((start >= stop) and (step > 0)) or ((start <= stop) and (step < 0)) or (step == 0):
        raise ValueError('you are loh')

    if start <= stop:
        while current_number < stop:
            yield current_number
            current_number += step
    else:
        while current_number > stop:
            yield current_number
            current_number += step

In [29]:
for i in float_range(start=1, stop=-1, step=-0.5):
    print(i)

1.0
0.5
0.0
-0.5


## Задание 2: свой map

### Часть 1: копируем map

Реализуйте аналог функции map, полностью копирующий ее поведение. Саму map использовать нельзя.

In [19]:
from typing import Iterable

def my_map(
    func, # Почему не дает задать через func: function????
    *args: Iterable
) -> Generator:

    for arg in zip(*args):
        yield func(*arg)

### Часть 2: дополняем map

Добавьте возможность управлять поведением вашего map'a: сделайте так, чтобы map имела возможность не только обрезать последовательности, но и дополнять короткие последовательности до динных. 

Совет: функция **zip_longest** из библиотеки **itertools** может оказаться полезной.

In [21]:
from enum import Enum
from itertools import zip_longest


class MapTypes(Enum):
    SHORTEST = 'short'
    LONGEST = 'long'


In [24]:
from typing import Iterable

def my_map(
    func, 
    type: MapTypes = MapTypes.SHORTEST,
    *args: Iterable
) -> Generator:

    if type==MapTypes.LONGEST:
        for arg in zip_longest(*args):
            yield func(*arg)

    else:
        for arg in zip(*args):
            yield func(*arg)

## Задание 3: Спиннер

### Часть 1: генератор

Напишите функцию, которая на вход получает коллекцию и возвращает генератор, последовательно возвращающий элементы коллекции, а после возврата последнего элемента коллекции очередной вызов генератора приведет к зацикливанию.


*Пример*:

```python
generator = generate_circle('abc')

print(next(generator))
print(next(generator))
print(next(generator))
print(next(generator))
print(next(generator))
```
*Вывод*:
```console
a
b
c
a
b
```

In [11]:
from typing import Generator

def generate_circle(
    input: Iterable
) -> Generator[chr, None, None]:

    while True:
        for elem in input:
            yield elem

In [37]:
generator = generate_circle('penis')

for i in range(10):
    print(next(generator))

p
e
n
i
s
p
e
n
i
s


### Часть 2: Колесо Сансары

Используя генератор из предыдущего раздела, реализуйте функцию, которая отображает на экране спиннер для индикации загрузки.

Отрисовка спиннера печатает на экран надпись: Thinking: \<symbol\>, где вместо \<symbol\> последовательно появляются знаки: \, |, /, -, что создаёт эффект вращения.

Вход функции: 
- time_limit - время (в секундах), в течение которого должна производиться отрисовка спиннера;
- pause - время (в секундах) задержки между сменой символов спиннера;

Интересная статья на тему индикаторов: https://dtf.ru/flood/174240-progress-bar-ili-spinner-chto-i-kogda-ispolzovat?ysclid=lorrg51syv550654720

In [14]:
from typing import Generator
from time import time

def generate_circles(
    time_limit: float,
    pause: float
) -> Generator[str, None, None]:

    time_limit, pause = float(time_limit), float(pause)
    generator = generate_circle("|/-\|/-" + chr(92))

    for i in float_range(stop = time_limit, step = pause):
        print(next(generator))

|
/
-
\
|
/
-
\
|
/
-
\
|
/
-
\
|
/
-
\
